In [ ]:
import pandas as pd
from datasets import load_dataset

# initial tests using sample transcripts of town hall meetings from MeetingBank
ds = load_dataset("huuuyeah/meetingbank")


In [ ]:
# split the dict
train_data = ds['train']
test_data = ds['test']
val_data = ds['validation']

ds

DatasetDict({
    train: Dataset({
        features: ['summary', 'uid', 'id', 'transcript'],
        num_rows: 5169
    })
    validation: Dataset({
        features: ['summary', 'uid', 'id', 'transcript'],
        num_rows: 861
    })
    test: Dataset({
        features: ['summary', 'uid', 'id', 'transcript'],
        num_rows: 862
    })
})

In [ ]:
# converting to dataframes
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)
val_df = pd.DataFrame(val_data)

In [13]:
train_df.head()

,summary,uid,id,transcript
0,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_05012017_17-0161,0,Please refrain from profane or obscene speech....
1,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_04102017_17-0161,1,An assessment has called out council bill 161 ...
2,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_02272017_17-0161,2,I Please close the voting. Announce the result...
3,Recommendation to respectfully request City Co...,LongBeachCC_03072017_17-0161,3,"Motion passes. Hey, thank you very much. Now w..."
4,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_03202017_17-0161,4,All right. Pursuant to Council Bill 3.7 consid...


In [14]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

In [15]:
# Load environment variables.
load_dotenv()

# Set the model name for our LLMs.
OPENAI_MODEL = "gpt-3.5-turbo"
# Store the API key in a variable.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [16]:
from langchain.agents import initialize_agent, load_tools
from langchain.chains.llm import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain

In [ ]:
# dropping entries with too many tokens for the model
# this was before batching was considered
train_df_sorted = train_df[train_df['transcript'].str.len() <= 16385 * 4].reset_index()
train_df_sorted

,index,summary,uid,id,transcript
0,1,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_04102017_17-0161,1,An assessment has called out council bill 161 ...
1,2,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_02272017_17-0161,2,I Please close the voting. Announce the result...
2,3,Recommendation to respectfully request City Co...,LongBeachCC_03072017_17-0161,3,"Motion passes. Hey, thank you very much. Now w..."
3,4,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_03202017_17-0161,4,All right. Pursuant to Council Bill 3.7 consid...
4,5,AS AMENDED a bill for an ordinance amending th...,DenverCityCouncil_02212017_17-0161,5,"11 Eyes, Resolution 109 has been adopted as am..."
...,...,...,...,...,...
4882,5164,AN ORDINANCE imposing a tax on engaging in the...,SeattleCityCouncil_06052017_CB 118965,5164,Thank you. The bill passed and chair of the Se...
4883,5165,Proclamation Declaring April 8 through April 1...,AlamedaCC_04022019_2019-6703,5165,Okay. Next is a proclamation declaring April e...
4884,5166,Approves an agreement with Kaiser Foundation H...,DenverCityCouncil_01052015_14-1062,5166,I just pulled this one out. This one actually ...
4885,5167,A bill for an Ordinance authorizing and approv...,DenverCityCouncil_06082015_15-0302,5167,The estimated taxable value following redevelo...


In [ ]:
# view a sample transcript
train_df_sorted['transcript'][2]

"Motion passes. Hey, thank you very much. Now we're moving on to item 22. Communication from Vice Mayor Richardson recommendation to request City Council approval of the 2017 federal legislative agenda as recommended by the Federal Legislation Committee. There's a motion and a second Vice Mayor. So just a few updates here we have. So we met in November on November 15th to review staff's recommended recommended changes for the 2017 federal legislative agenda. The committee made some fairly substantive changes to the agenda last year when we were reorganized and consolidated some of the statements that may have been repetitive in years past. And because we made so many, so many changes in the last update, and as the new presidential administration gets up and running, the first large committee will be focused on protecting existing revenues that the city current receive currently receives from the federal government, such as the Housing Choice Voucher Program, commonly known as Section e

In [ ]:
# pulling in zains real world transcript
transcript = pd.read_json('../Resources/vertopal.com_Discuss Collaborative Testing prior to release.json', orient='index')
transcript

0
﻿Transcript\nApril 29, 2025, 2:02PM\n\nZain Mas... NaN

In [20]:
'''#audio transcription bot
from openai import OpenAI

client = OpenAI()
audio_file = open("Discuss RCM _ CPQ Data Usecases-20250411_081350-Meeting Recording.mp3", 'rb')

transcribed_audio = client.audio.transcriptions.create(
    file=audio_file,
    model='whisper-1',
    response_format='text'
)
transcribed_audio = '"' + transcribed_audio + '"'
print(transcribed_audio)'''

'#audio transcription bot\nfrom openai import OpenAI\n\nclient = OpenAI()\naudio_file = open("Discuss RCM _ CPQ Data Usecases-20250411_081350-Meeting Recording.mp3", \'rb\')\n\ntranscribed_audio = client.audio.transcriptions.create(\n    file=audio_file,\n    model=\'whisper-1\',\n    response_format=\'text\'\n)\ntranscribed_audio = \'"\' + transcribed_audio + \'"\'\nprint(transcribed_audio)'

## adjust comment code cells above or below depending on test input

In [21]:
# calling it the same variable for simplicity later - comment out if using audio

transcribed_audio = str(transcript[0])

In [22]:
len(transcribed_audio)

56998

In [23]:
# splitting transcription if max token limit is possibly exceeded
max_len = 37000

if len(transcribed_audio) >= max_len:

    transcribed_audio_trimmed_list = [transcribed_audio[i:i + max_len] for i in range(0, len(transcribed_audio), max_len)]

transcribed_audio_trimmed_list

["\ufeffTranscript\\nApril 29, 2025, 2:02PM\\n\\nZain Master started transcription\\n\\nZain Master   0:03\\nEverybody, the purpose of this meeting is to talk about and understand and formulate plans on how to make sure that we can increase the confidence of our.\\nReleases going out to customers, right? As we understand most of our products are impacted by a level of customizations that involve very specific customer setups and implementations.\\nSo while the product is able to perform a level of testing, there needs to be that next level of testing that is performed in specific to that customer's implementation and that specific environment. Many times as we've seen, we've gotten a release out there only to have it.\\nImpacted either by product score functionality that needs to be compatible with a implementation, or the implementation side of the House.\\nThat's incompatible with product and then that gets us into this wonderful spiral of trying to continually address.\\nThe issue a

# the following models will take in either the transcribed audio, or a transcribed text document!

In [ ]:
#summary bot

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name='gpt-4-turbo', temperature=0)


prompt = ChatPromptTemplate.from_messages(
    [("system", "Write a detailed summary of the following:\\n\\n{text}")]
)

# Instantiate chain
chain = load_summarize_chain(llm, chain_type="stuff", prompt=prompt)

# Create Document objects from the filtered transcripts
documents = transcribed_audio_trimmed_list

docs = [Document(page_content=doc) for doc in documents]


# Batching the docs if they're too long
def batch_documents(docs, batch_size=1):
    for i in range(0, len(docs), batch_size):
        yield docs[i:i + batch_size]


batch_summaries = []
for batch in batch_documents(docs):
    combined_text = '\n\n'.join([doc.page_content for doc in batch])
    summary = chain.run(input_documents=batch)
    batch_summaries.append(summary)

# summarize all summaries
final_input_docs = [Document(page_content=summary) for summary in batch_summaries]
final_summary = chain.run(input_documents=final_input_docs)

print(final_summary)

C:\Users\Nik\AppData\Local\Temp\ipykernel_2012\4008238935.py:27: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  summary = chain.run(input_documents=batch)


The transcript from April 29, 2025, captures a critical meeting led by Zain Master, focusing on enhancing customer confidence in product releases. The primary concern addressed is the need for improved testing methodologies that closely mirror the unique environments of individual customers. This approach is aimed at mitigating the recurring issues that necessitate continuous fixes, which have been a source of dissatisfaction among customers and internal teams alike.

Zain proposes a collaborative strategy between the solution center, which has a deep understanding of customer implementations, and the core product team responsible for initial testing phases. This partnership is envisioned to enhance the effectiveness of the testing processes by incorporating insights from the solution center into the core team's efforts.

The meeting also delves into the strategy of releasing updates in waves. This method has proven beneficial in early identification of issues by limiting the exposure 

In [25]:
from openai import OpenAI

# Call to action bot
prompt2 = """You are an intelligent assistant tasked with extracting all unique calls to action from the following text. Return each call to action seperately with a newline character. For example:

-Call to Action 1
-Call to Action 2
-Call to Action 3
...

Here is the text:
"""
client = OpenAI()
# Instantiate chain
chain2 = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[{'role': 'system', 'content': prompt2},
              {'role': 'user', 'content': transcribed_audio}]
)

calls = chain2.choices[0].message.content
print(calls)

-Generate and store output for integrations prior to every release to compare
-Develop a train-the-trainer model for disseminating information within the organization
-Include Solution Center resources in demo meetings to stay updated on customization work
-Formalize the process for booking QA resources for testing prior to releases
-Implement API performance testing as part of the testing scope
-Improve management of accesses to connectors and two-factor authentication for better confidence
-Plan API performance testing for the roadmap and schedule it accordingly.


In [26]:
meeting_results = {'summary': final_summary, 'action calls': calls}

meeting_results['summary'] = 'Summary:\n\n' + meeting_results['summary'] + '\n'
meeting_results['action calls'] = ('Calls to Action:\n' + meeting_results['action calls']).replace('\n', '\n\n')

print(meeting_results['summary'])
print(meeting_results['action calls'])

Summary:

The transcript from April 29, 2025, captures a critical meeting led by Zain Master, focusing on enhancing customer confidence in product releases. The primary concern addressed is the need for improved testing methodologies that closely mirror the unique environments of individual customers. This approach is aimed at mitigating the recurring issues that necessitate continuous fixes, which have been a source of dissatisfaction among customers and internal teams alike.

Zain proposes a collaborative strategy between the solution center, which has a deep understanding of customer implementations, and the core product team responsible for initial testing phases. This partnership is envisioned to enhance the effectiveness of the testing processes by incorporating insights from the solution center into the core team's efforts.

The meeting also delves into the strategy of releasing updates in waves. This method has proven beneficial in early identification of issues by limiting the

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

# test query bot

results = [transcribed_audio, meeting_results['summary'], meeting_results['action calls']]

embeddings = SentenceTransformerEmbeddings(model_name='all-mpnet-base-v2')

docsearch = FAISS.from_texts(
    texts=results,
    embedding=embeddings
)

llm3 = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=OPENAI_MODEL, temperature=0)

qa = RetrievalQA.from_chain_type(
    llm=llm3,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={'k':1}),
    return_source_documents=False
)


query = 'What is the summary for this meeting?'

response = qa.invoke(query)
print('Summary:\n')
print(response['result'] + '\n')

query_cta = 'What are the calls to action?'

response_cta = qa.invoke(query_cta + ' Return output in bullet points')
print('Calls to Action:\n')
print(response_cta['result'].replace('\n', '\n\n') + '\n')


query_any = 'Can you provide more context on the concerns of the meeting?'

response_any = qa.invoke(query_any)
print('Context Result:\n')
print(response_any['result'].replace('\n', '\n\n'))

C:\Users\Nik\AppData\Local\Temp\ipykernel_32236\653341844.py:8: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name='all-mpnet-base-v2')



Summary:

The meeting discussed various topics related to improving the testing process for releases, focusing on increasing confidence in releases and addressing issues related to customer implementations, API integrations, and performance testing. Key points included the need for cross-collaboration between Solution Center and Core Product teams, the importance of testing integrations and APIs, and the necessity of improving access management for connectors. The meeting also highlighted the significance of proactive participation from Solution Center in understanding core functionality changes and the need for a structured process to ensure effective testing and communication between teams.

Calls to Action:

- Generate and store output for integrations prior to every release to compare

- Start tracking and comparing API performance testing results

- Implement a train-the-trainer model for disseminating information about changes to integrations and outputs

- Schedule a demo at Pu

In [ ]:
# reading in tasks and completion time training data
capacity = pd.read_csv('../Resources/CommonTasks.csv')

capacity1 = capacity[['Common Tasks Types', 'Average Task duration in minutes']]
list_capacity = capacity1.values.tolist()

In [ ]:
# bot trained to label CTAs with avg completion time
cta_time_prompt = """You are a smart and intelligent project manager that understand how much time it could take on average to complete an action item.
You will take in a meeting transcript, and a table of common tasks and their estimated completion time in minutes.
Based on all of the action items/calls from the uploaded meeting transcript, you will reference the table of common tasks and assign an approximate time value in minutes.
If a task is hard for you to assign a time to complete, list the task followed by 'Unable to estimate'
For every task to do, list the task followed by the duration on the same continuous line.
Your output should be a list of every task from the list, and their average time to complete.
Ensure each task entry and their duration to complete are one continuous line.
An Example Output List:

Task 1: (# Mins)
Task 2: (# Mins)
Task 3: (# Mins)
Task X: (Unable to estimate).
"""


client = OpenAI()
chain3 = client.chat.completions.create(
model='gpt-4-turbo',
messages=[{'role': 'system', 'content': cta_time_prompt},
            {'role': 'system', 'content': f"Task 1: {str(calls)}\n Duration: {str(list_capacity)}\n"}]
)

cta_times_list = chain3.choices[0].message.content
cta_times_list = cta_times_list.strip('\n\n')

print(cta_times_list)

- Generate and store output for integrations prior to every release to compare: (Setup API to store output: 60 mins)
- Develop a train-the-trainer model for disseminating information within the organization: (Training others on job: 90 mins)
- Include Solution Center resources in demo meetings to stay updated on customization work: (Provide Demo: 60 mins)
- Formalize the process for booking QA resources for testing prior to releases: (Refine an existing process: 1200 mins)
- Implement API performance testing as part of the testing scope: (API performance testing: 2400 mins)
- Improve management of accesses to connectors and two-factor authentication for better confidence: (Unable to estimate)
- Plan API performance testing for the roadmap and schedule it accordingly: (API performance testing: 2400 mins)


In [ ]:
# example members dictionary test input
names_dict = {
    "Petr Nesladek": "petr.nesladek@example.com",
    "Petr Klen": "petr.klen@example.com",
    "Terry Stone": "terry.stone@example.com",
    "Radim Moravec": "radim.moravec@example.com",
    "Lukas Kawulok": "lukas.kawulok@example.com",
    "Rene Gruszkowski": "rene.gruszkowski@example.com",
    "Stephen Pack": "stephen.pack@example.com",
    "Gloria Kee": "gloria.kee@example.com",
    "Jiri Sverepa": "jiri.sverepa@example.com",
    "Chintan Modha": "chintan.modha@example.com",
    "Petr Lofferman": "petr.lofferman@example.com"
}

In [ ]:
# grabbing the names to check for matches in transcript
names_dict_list = list(names_dict.keys())
names_dict_list

['Petr Nesladek',
 'Petr Klen',
 'Terry Stone',
 'Radim Moravec',
 'Lukas Kawulok',
 'Rene Gruszkowski',
 'Stephen Pack',
 'Gloria Kee',
 'Jiri Sverepa',
 'Chintan Modha',
 'Petr Lofferman']

In [42]:
# NER bot
prompt3 = """You are a smart and intelligent Named Entity Recognition (NER) system. You will take in a list of possible Names that attended a meeting, and Identify all of the unique matching names that appear in the given meeting transcription.
Importantly, your output should have each name separate with a newline character. For example:

First Name Last Name 1
First Name Last Name 2
First Name Last Name 3
...

Here is the text:
"""

# Instantiate chain
chain3 = client.chat.completions.create(
    model=OPENAI_MODEL,
    messages=[{'role': 'system', 'content': prompt3},
              {'role': 'user', 'content': f'Names list: {names_dict_list}\n Meeting Transcription {transcribed_audio}'}]
)

names = chain3.choices[0].message.content
print(names)

Petr Nesladek
Petr Klen
Terry Stone
Radim Moravec
Lukas Kawulok
Rene Gruszkowski
Stephen Pack
Gloria Kee
Jiri Sverepa
Chintan Modha


In [ ]:
# convert pulled names to list
names_list = names.split('\n')
names_list

['Terry Stone',
 'Petr Klen',
 'Stephen Pack',
 'Zain Master',
 'Lukas Kawulok',
 'Radim Moravac']

In [ ]:
# fuzzy match possible names with names pulled from LLM to account for LLM spelling errors

from rapidfuzz import fuzz, process

names = ['Terry Stone', 'Petr Klen', 'Stephen Pack', 'Zain Master', 'Lukas Kawulok', 'Radim Moravec', 'Petr Nesladek']

threshold = 70
matched_names = []

for name in names_dict.keys():
    match, score, _ = process.extractOne(name, names_list, scorer=fuzz.token_sort_ratio)
    if score >= threshold:
        matched_names.append(name)

matched_names

['Terry Stone',
 'Petr Klen',
 'Stephen Pack',
 'Zain Master',
 'Lukas Kawulok',
 'Radim Moravac']

In [ ]:
# filter input dict to drop entires that were not matched
filtered_members = {k: v for k, v in names_dict.items() if k in matched_names}
filtered_members

{'Terry Stone': 'terry.stone@example.com',
 'Petr Klen': 'petr.klen@example.com',
 'Stephen Pack': 'stephen.pack@example.com',
 'Zain Master': 'zain.master@example.com',
 'Lukas Kawulok': 'lukas.kawulok@example.com',
 'Radim Moravac': 'radim.moravec@example.com'}

In [ ]:
# convert to dataframe
members_info = pd.DataFrame({'member': filtered_members.keys(), 'email': filtered_members.values()})
members_info

,member,email
0,Terry Stone,terry.stone@example.com
1,Petr Klen,petr.klen@example.com
2,Stephen Pack,stephen.pack@example.com
3,Zain Master,zain.master@example.com
4,Lukas Kawulok,lukas.kawulok@example.com
5,Radim Moravac,radim.moravec@example.com


In [ ]:
# create list of identified ctas
meeting_results_action_list = list(meeting_results['action calls'])

In [ ]:
# NER bot 2 - matches ctas to names
prompt4 = """You are a highly accurate Named Entity Recognition (NER) system.  
Your task is to analyze a meeting transcription along a list of Calls to Action (CTAs) and identify Calls to Action (CTAs) directed at individuals from a provided list of names.

Instructions:
1. For each name in the list, check if a unique task or action was assigned or implied in the transcription.
2. If a task exists, return the name followed by the task — all in one line.
3. If no task exists for a name, return the name followed by: "There were no calls to action directed towards you."

⚠️ Ensure:
- Every name from the input list is accounted for in the output — no omissions.
- The output is structured as a clean list: one name per line, followed by the corresponding task or message.

Example Output Format:
Name 1: Follow up with the client on the pricing details.
Name 2: Prepare the Q2 financial report.
Name 3: There were no calls to action directed towards you.

Begin processing now."""

# Instantiate chain
chain4 = client.chat.completions.create(
    model='gpt-4-turbo',
    messages=[{'role': 'system', 'content': prompt4},
              {'role': 'user', 'content': f"Names: {str(matched_names)}\n Meeting: {transcribed_audio}\n Calls to Action {meeting_results_action_list}"}]
)

names_calls = chain4.choices[0].message.content
name_calls = names_calls.strip('\n \n')
print(names_calls)

Terry Stone: There were no calls to action directed towards you.
Petr Klen: There were no calls to action directed towards you.
Stephen Pack: Capture the process of reaching out for resources to formalize it.
Zain Master: Generate and store output for integrations prior to every release to compare.
Lukas Kawulok: There were no calls to action directed towards you.
Radim Moravac: There were no calls to action directed towards you.


In [ ]:
# split returned strings by Member: and cta
import re

delimiters = [r": ", "\n"]
pattern = "|".join(map(re.escape, delimiters))

names_calls_list = re.split(pattern, names_calls)
names_calls_list

['Terry Stone',
 'There were no calls to action directed towards you.',
 'Petr Klen',
 'There were no calls to action directed towards you.',
 'Stephen Pack',
 'Capture the process of reaching out for resources to formalize it.',
 'Zain Master',
 'Generate and store output for integrations prior to every release to compare.',
 'Lukas Kawulok',
 'There were no calls to action directed towards you.',
 'Radim Moravac',
 'There were no calls to action directed towards you.']

In [ ]:
# converting split results into dataframe
meeting_actions = {}
i = 0
while i < len(names_calls_list) - 1:
    name = names_calls_list[i].strip()
    action = names_calls_list[i + 1].strip()
    meeting_actions[name] = action
    i += 2

meeting_actions = pd.DataFrame.from_dict([meeting_actions]).T.reset_index().rename(columns={'index': 'member', 0: 'action'})
meeting_actions

,member,action
0,Terry Stone,There were no calls to action directed towards...
1,Petr Klen,There were no calls to action directed towards...
2,Stephen Pack,Capture the process of reaching out for resour...
3,Zain Master,Generate and store output for integrations pri...
4,Lukas Kawulok,There were no calls to action directed towards...
5,Radim Moravac,There were no calls to action directed towards...


In [ ]:
# checking if names match
meeting_actions.info

<bound method DataFrame.info of           member                                             action
0    Terry Stone  There were no calls to action directed towards...
1      Petr Klen  There were no calls to action directed towards...
2   Stephen Pack  Capture the process of reaching out for resour...
3    Zain Master  Generate and store output for integrations pri...
4  Lukas Kawulok  There were no calls to action directed towards...
5  Radim Moravac  There were no calls to action directed towards...>

In [ ]:
# checking if names match
members_info.info

<bound method DataFrame.info of           member                      email
0    Terry Stone    terry.stone@example.com
1      Petr Klen      petr.klen@example.com
2   Stephen Pack   stephen.pack@example.com
3    Zain Master    zain.master@example.com
4  Lukas Kawulok  lukas.kawulok@example.com
5  Radim Moravac  radim.moravec@example.com>

In [ ]:
# another fuzzy match to fix spelling of names from cta-names match LLM if need be
info = list(members_info['member'].values)

actions = list(meeting_actions['member'].values)


threshold = 70

matched_names_actions = []

for name in info:
    match, score, _ = process.extractOne(name, actions, scorer=fuzz.token_sort_ratio)
    if score >= threshold:
        matched_names_actions.append(name)

matched_names_actions

['Terry Stone',
 'Petr Klen',
 'Stephen Pack',
 'Zain Master',
 'Lukas Kawulok',
 'Radim Moravac']

In [ ]:
# overwriting with correct spelling
meeting_actions['member'] = matched_names_actions

In [ ]:
# merging names - ctas with info
meeting_actions_emails = pd.merge(meeting_actions, members_info, on='member', how='outer')
meeting_actions_emails

,member,action,email
0,Lukas Kawulok,There were no calls to action directed towards...,lukas.kawulok@example.com
1,Petr Klen,There were no calls to action directed towards...,petr.klen@example.com
2,Radim Moravac,There were no calls to action directed towards...,radim.moravec@example.com
3,Stephen Pack,Capture the process of reaching out for resour...,stephen.pack@example.com
4,Terry Stone,There were no calls to action directed towards...,terry.stone@example.com
5,Zain Master,Generate and store output for integrations pri...,zain.master@example.com


In [45]:
str(meeting_results)

"{'summary': 'Summary:\\n\\nThe transcript from April 29, 2025, outlines a critical meeting led by Zain Master, focusing on enhancing customer confidence in product releases. The primary concern addressed is the challenge of customer-specific customizations and environments, which have historically led to compatibility issues, resulting in a repetitive cycle of releases and fixes. This cycle has caused frustration both within the company and among customers.\\n\\nZain Master proposes a strategy aimed at fostering better collaboration between the solution center and the core product team. The solution center has a deep understanding of customer implementations, while the core product team is responsible for initial testing. By integrating these teams, the company aims to ensure that testing is thorough and that updates do not lead to significant issues post-release.\\n\\nA significant part of the strategy involves releasing updates in waves. This approach allows for smaller-scale initia

In [ ]:
# Email bot
prompt5 = """You are an intelligent Email formatting Assistant. You will take in a summary and calls to action from a meeting transcript, as well as a name, with their corresponding specific action and Email address.
Your task is to write this person an email. Fill in the To: field with their email, output the summary, followed by all calls to actions. Then the corresponding specific action(s) for the member the Email is written for. An example Email Format is as follows:

To: sarah.chen@example.com

From: znpmeetingassistant@gmail.com

Subject: Project Phoenix - Kickoff Meeting Summary

This email summarizes the kickoff meeting for Project Phoenix, held on October 26, 2023.

! Warning ! Assignments of tasks and Summaries generated by this Bot may be innacurate. Please be sure to follow up with Meeting Organizer / Team Lead

Meeting Summary:

The meeting began with an overview of Project Phoenix's objectives: to develop and launch a new customer relationship management (CRM) system within the next six months. We discussed the project's scope, which includes migrating existing customer data, integrating with our current marketing automation platform, and training our sales and support teams on the new system.

Key discussion points included:

- Data Migration Strategy: We reviewed the proposed approach for migrating customer data from the legacy system to the new CRM.

- Integration Requirements: We discussed the necessary integrations with the marketing automation platform, focusing on data synchronization and workflow automation.

- Training Plan: The training team outlined the plan for developing and delivering training materials to sales and support staff.

- Timeline and Milestones: We established the initial project timeline, identifying key milestones and deadlines. A detailed project schedule will be shared next week.

- Budget Allocation: Initial budget was presented.

Calls to Action:

- The following actions were assigned to ensure the project stays on track:

- Sarah Chen: Finalize the data migration plan and submit it for review by November 2nd.

- John Smith: Define the API specifications for the marketing automation platform integration by November 5th.

- Emily Brown: Develop the initial training modules outline by November 9th.

- David Lee: Circulate the detailed project schedule by October 28th.

- Michael Davis: Provide a detailed budget breakdown by November 2nd.


Specific Call to Action for Sarah Chen:

Sarah, your immediate action is to finalize the data migration plan, including the specific steps, resources required, and potential risks. Please submit this plan to project_team@example.com by November 2nd.

Thank you for your contributions to the meeting. Please let me know if you have any questions or require further clarification.

Best regards,

ZNP Meeting Assistant*

* ! Warning ! Assignments of tasks and Summaries generated by this Bot may be innacurate. Please be sure to follow up with Meeting Organizer / Team Lead



Here is the information:
"""
# loops through each entry to generate email for every member
def generate_emails(meeting_results, meeting_actions_emails):
    emails = []
    for _, row in meeting_actions_emails.iterrows():
        member_name = row['member']
        member_email = row['email']
        specific_action = row['action']

        # Construct the user prompt with the specific information for the member
        user_prompt = f"""
        Here is the meeting summary:
        {meeting_results}

        Here are the calls to action for all members:
        {meeting_actions_emails.to_string(index=False)}

        Here is the specific call to action for {member_name}:
        {specific_action}
        """

        messages = [
            {'role': 'system', 'content': prompt5},
            {'role': 'user', 'content': user_prompt}
        ]

        try:
            chain = client.chat.completions.create(
                model=OPENAI_MODEL,
                messages=messages
            )
            email_content = chain.choices[0].message.content
            emails.append(email_content) 
            print(f"Generated email for {member_name} ({member_email})")
        except Exception as e:
            print(f"An error occurred while generating email for {member_name}: {e}")
            emails.append(f"Error: Could not generate email for {member_name}.")

    return emails


# Generate and store emails
generated_emails = generate_emails(meeting_results, meeting_actions_emails)

# Print the list of generated emails
print("\n--- Generated Emails ---")
for email in generated_emails:
    print(email)

Generated email for Lukas Kawulok (lukas.kawulok@example.com)
Generated email for Petr Klen (petr.klen@example.com)
Generated email for Radim Moravac (radim.moravec@example.com)
Generated email for Stephen Pack (stephen.pack@example.com)
Generated email for Terry Stone (terry.stone@example.com)
Generated email for Zain Master (zain.master@example.com)

--- Generated Emails ---
To: lukas.kawulok@example.com

From: znpmeetingassistant@gmail.com

Subject: Summary and Action Items from April 29, 2025 Meeting

Hi Lukas,

I hope this email finds you well. Below is a summary of the meeting held on April 29, 2025, focusing on enhancing customer confidence in product releases:

Meeting Summary:

The meeting, led by Zain Master, highlighted the challenges related to customer-specific customizations and compatibility issues post-release. A strategy was proposed to integrate the solution center and core product team for thorough testing and implementing wave-based release updates to mitigate disru

In [47]:
len(generated_emails)

6

In [48]:
print(generated_emails[1])

To: petr.klen@example.com

From: znpmeetingassistant@gmail.com

Subject: Summary and Calls to Action from April 29, 2025 Meeting

This email summarizes the critical meeting held on April 29, 2025, focusing on enhancing customer confidence in product releases.

Meeting Summary:

The meeting, led by Zain Master, addressed the challenges posed by customer-specific customizations and environments, aiming to prevent compatibility issues and the repetitive cycle of releases and fixes. The proposed strategy involves fostering collaboration between the solution center and the core product team, releasing updates in waves to ensure thorough testing and protect major customers from disruptions. The importance of communication, resource allocation, and an Agile framework for effective software development was highlighted.

Calls to Action:

- Generate and store output for integrations prior to every release to compare.
- Schedule another meeting to discuss performance testing and API integrations

In [ ]:
# since the output is carefully curated, we can extract the information needed for email sending
# splitting each email into a list
email_bones = []
for email in generated_emails:
    lines = email.splitlines()
    email_bones.append(lines)

email_bones[0]

['To: lukas.kawulok@example.com',
 '',
 'From: znpmeetingassistant@gmail.com',
 '',
 'Subject: Summary and Action Items from April 29, 2025 Meeting',
 '',
 'Hi Lukas,',
 '',
 'I hope this email finds you well. Below is a summary of the meeting held on April 29, 2025, focusing on enhancing customer confidence in product releases:',
 '',
 'Meeting Summary:',
 '',
 'The meeting, led by Zain Master, highlighted the challenges related to customer-specific customizations and compatibility issues post-release. A strategy was proposed to integrate the solution center and core product team for thorough testing and implementing wave-based release updates to mitigate disruptions for key customers.',
 '',
 'Key Action Items:',
 '',
 '- Capture the process of reaching out for resources to formalize it.',
 '',
 'Specific Call to Action for Lukas Kawulok:',
 '',
 "Lukas, while there were no specific actions assigned to you during this meeting, please continue your valuable contributions to the team's

In [ ]:
# looping through each email list to extract email sending information
# these variables are looped through to send each email
to_list = []
for email in email_bones:
    to_list.append(email[0].replace('To: ', ''))

sender_list = []
for email in email_bones:
    sender_list.append(email[2].replace('From: ', ''))

subject_list = []
for email in email_bones:
    subject_list.append(email[4].replace('Subject: ', ''))

message_text_list = []
for email in email_bones:
    message_text_list.append(str(email[6:]).replace("''", '\n').replace("',", '\n').replace("\n,", '').replace('\n', '\n\n').replace('",', '\n\n').replace('"', '').replace("'", "").replace('[', '\n').replace(']', '\n'))


'''to = email_bones[0]
cc = email_bones[2]
sender = email_bones[4]
subject = email_bones[6]
message_text = str(email_bones[8:]).replace("''", '\n').replace("',", '\n').replace("\n,", '').replace('\n', '\n\n').replace('",', '\n\n').replace('"', '').replace("'", "").replace('[', '\n').replace(']', '\n')'''


print(to_list[0])
print(sender_list[0])
print(subject_list[0])
print(message_text_list[0])


lukas.kawulok@example.com
znpmeetingassistant@gmail.com
Summary and Action Items from April 29, 2025 Meeting

Hi Lukas,

  I hope this email finds you well. Below is a summary of the meeting held on April 29, 2025, focusing on enhancing customer confidence in product releases:

  Meeting Summary:

  The meeting, led by Zain Master, highlighted the challenges related to customer-specific customizations and compatibility issues post-release. A strategy was proposed to integrate the solution center and core product team for thorough testing and implementing wave-based release updates to mitigate disruptions for key customers.

  Key Action Items:

  - Capture the process of reaching out for resources to formalize it.

  Specific Call to Action for Lukas Kawulok:

  Lukas, while there were no specific actions assigned to you during this meeting, please continue your valuable contributions to the teams initiatives in creating a seamless collaboration between the solution center and the core

In [58]:
import base64
from email.message import EmailMessage

In [59]:
import os.path
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

In [60]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders

In [ ]:
# load gmail password
load_dotenv()

SENDER_PASSWORD = os.getenv('SENDER_PASSWORD')

In [ ]:
# Set SMPT variables
sender_email = "znpmeetingassistant@gmail.com"
sender_password = SENDER_PASSWORD
sender_password = sender_password.replace('-', ' ')
smtp_server = "smtp.gmail.com" 
smtp_port = 587

In [ ]:
# create and send email functions
def create_email(receiver_email, subject, message_text):
    message = MIMEMultipart()
    message['From'] = sender_email
    message['To'] = receiver_email
    message['Subject'] = subject
    message.attach(MIMEText(message_text, 'plain'))
    return message.as_string()



def send_email(receiver_email, subject, message_text):
    message = create_email(receiver_email, subject, message_text)

    try:
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            server.sendmail(sender_email, receiver_email, message)
        print(f"Email sent successfully to {receiver_email}")
    except Exception as e:
        print(f"Error sending email: {e}")

# loops through each email info list to create and send emails for every member
def send_bulk_email(receiver_list, subject_list, message_list):
    for receiver, subject, message in zip(receiver_list, subject_list, message_list):
        send_email(receiver, subject, message)



In [ ]:
# testing the function
send_bulk_email(to_list, subject_list, message_text_list)

Email sent successfully to lukas.kawulok@example.com
Email sent successfully to petr.klen@example.com
Email sent successfully to radim.moravec@example.com
Email sent successfully to stephen.pack@example.com
Email sent successfully to terry.stone@example.com
Email sent successfully to zain.master@example.com


In [ ]:
# old gmail API access code, works but is more cumbersome to use

'''SCOPES = ['https://www.googleapis.com/auth/gmail.send']


def create_message(sender, to, subject, message_text):
    message = EmailMessage()
    message.set_content(message_text)
    message['To'] = to
    message['From'] = sender
    message['Subject'] = subject
    return {'raw': base64.urlsafe_b64encode(message.as_bytes()).decode()}


def send_message(service, user_id, message):
    try:
        message = (service.users().messages().send(userId=user_id, body=message)
                 .execute())
        print('Message Id: %s' % message['id'])
        return message
    except Exception as error:
        print(f'An error occurred: {error}')

def email_main(to_list, subject_list, message_text_list,):
    creds = None
    if os.path.exists('token.json'):
        creds = Credentials.from_authorized_user_file('token.json', SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=60010)
        with open('token.json', 'w') as token:
            token.write(creds.to_json())
    try:
        service = build('gmail', 'v1', credentials=creds)
        sender = "znpmeetingassistant@gmail.com"
        for to, subject, message_text in zip(to_list, subject_list, message_text_list):
            message = create_message(sender, to, subject, message_text)
            send_message(service, "me", message)
    except Exception as e:
        print(f"Error: {e}")'''

In [ ]:
#email_main(to_list, subject_list, message_text_list)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1092722805024-tu2g85irn4q8b7ap26lon70i902l0i73.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A60010%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.send&state=PUy1uUt9PXO3EtvvVKobIZVGa1VX8C&access_type=offline
